In [ ]:
#import relevant libraries 
import pandas as pd
import plotly.express as px
import panel as pn
import hvplot.pandas
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta,date
from pandas import DataFrame



In [ ]:
## Get Market Data for S&P500 
#engage API keys by activating .env file for Alpaca Api
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

#function to read the api data for stock ticker
#returns dataframe of closing price and daily returns for a given ticker symbol
def get_ticker_data(api,ticker):
    #load in historical data for provided ticker
    stock_data_df = api.alpha_vantage.historic_quotes(ticker, adjusted=True, output_format='pandas')

    #Clean Data
    

    #Sort earliest to latest. so that .pct_change() function works right.
    stock_data_df.sort_index(inplace=True, ascending=True)

    # Drop nulls
    stock_data_df.dropna(inplace=True)

    # drop duplicates
    stock_data_df.drop_duplicates(inplace=True)

    #count nulls 
    stock_data_df.isnull().sum()

    #create a dataframe column for the daily returns (pct_change) values and concat 
    returns_df = stock_data_df['5. adjusted close'].pct_change()
    stock_data_df = pd.concat([stock_data_df, returns_df], axis="columns", join="inner")

    #Change column names to avoid confusion
    columns = ['Open','High','Low','Close','Adjusted Close','Volume','Dividend Amount','Split Coefficient','Daily Returns']
    stock_data_df.columns = columns

    # Drop nulls
    stock_data_df.dropna(inplace=True)

    #drop duplicates
    stock_data_df.drop_duplicates(inplace=True) 
    return stock_data_df

In [1]:
stock_data_df=get_ticker_data(api,'SPY')

NameError: name 'get_ticker_data' is not defined

In [2]:
# function to get the MACD Value of the returns
# return MACD signal values dataframe
def get_MACD(stock_data_df,ticker, span1, span2,span3): ##typically span = 12, 26, 9
    # Grab just the `date` and `close` from the dataset
    macd_signals = stock_data_df.loc[:, ['Close']].copy()

    
    exp1 = macd_signals.Close.ewm(span1, adjust=False).mean()
    exp2 = macd_signals.Close.ewm(span2, adjust=False).mean()
    macd = exp1-exp2
    macd_out = macd.ewm(span3, adjust=False).mean()
    
    macd_signals["Signal"] = np.where(macd > macd_out, 1.0, 0.0)
    macd_signals["MACD Entry/Exit"] = mac_signals["Signal"].diff()

    return macd_signals


In [7]:
class MACDSimulator:
    def _init_(self, stock_data_df, ticker, span1, span2, span3, figsize = None):
        self.stock_data_df = stock_data_df
        self.ticker = ticker
        self.span1 = span1
        self.span2 = span2
        self.span3 = span3
        self.figsize = figsize

    def _build_dataframe(self):
        self.stock_data_df.sort_index(ascending=True, inplace=True)
    
    def _macd(self):
        self.stock_data_df['exp1'] = self.stock_data_df.Close.ewm(span1, adjust=False).mean()
        self.stock_data_df['exp2'] = self.stock_data_df.Close.ewm(span2, adjust=False).mean()
        self.stock_data_df['macd'] = exp1-exp2
        macd_out = macd.ewm(span3, adjust=False).mean()

    def _calculate_macd_signals(self):
        self.stock_data_df['Signal'] = None

        self.stock_data_df["Signal"] = np.where(self.stock_data_df.macd > macd_out, 1.0, 0.0, -1.0)
    self.stock_data.df["MACD Entry/Exit"] = self.stock_data.df["Signal"].diff()


        


NameError: name 'self' is not defined

In [3]:
def macd_plt(macd_signals, stock_data):

    plt.plot(stock_data.Close.ds, macd_signals)
    plt.plot(stock_data.Close.ds, exp3)
    plt.legend(loc='upper left')
    plt.show()


In [ ]:
exp1 = df.y.ewm(span=12, adjust=False).mean()
exp2 = df.y.ewm(span=26, adjust=False).mean()
exp3 = df.y.ewm(span=9, adjust=False).mean()
macd = exp1-exp2
plt.plot(df.ds, df.y)
plt.plot(df.ds, macd, label='AMD MACD', color='orange')
plt.plot(df.ds, exp3, label='Signal Line', color='Magenta')
plt.legend(loc='upper left')
plt.show()
